This notebook shows how to use a MIDI file to train and RNN and to seed the RNN for the generative stage.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys, os, re
%matplotlib inline

In [2]:
# for model
from keras.models import *
from keras.layers.core import *
from keras.layers.recurrent import *

# custom
from data_init import *
from model_util import *

#
import os.path
import random
from datetime import datetime

Using TensorFlow backend.


In [3]:
# ================================================================================
#
#
target_str = ""                         #
TARGET_FOLDER = "waltzes"               # training folder name
path_train = "./train_data/" + TARGET_FOLDER

# init paths of Waltzes
DIR_WEIGHTS = "./wts/"                      # save weights file
DIR_RESULTS = "./predict/"                  # save debug log
DIR_PREDICTED_MIDI = "./created/"          # save predicted(created) midi file

# file name to save
filename_result_predict = DIR_RESULTS + 'predict_{0}.txt'.format(datetime.now().strftime("%Y.%m.%d.%H:%M:%S"))

In [4]:
# directory for saving results and weights

if not os.path.exists(DIR_WEIGHTS):
    print "\n...... Make Folder ...... : ", DIR_WEIGHTS
    os.mkdir(DIR_WEIGHTS)
if not os.path.exists(DIR_RESULTS):
    print "\n...... Make Folder ...... : ", DIR_RESULTS
    os.mkdir(DIR_RESULTS)
if not os.path.exists(DIR_PREDICTED_MIDI):
    print "\n...... Make Folder ......", DIR_PREDICTED_MIDI
    os.mkdir(DIR_PREDICTED_MIDI)

In [5]:
##
#
# data set configuration
#

# example : target_str = "reels"

print "\n...... Get X, Y, samples_length, seq_length, hidden_size ...... : \n"
print "\tfrom {0},".format(path_train)
print "\ttarget_string : {0}\n".format(target_str)

MidiUtil = Midi_Util()                             
# X, Y, samples_length, seq_length, hidden_size = MidiUtil.get_data_set_from_midi(path_train)
X, Y, samples_length, seq_length, hidden_size = MidiUtil.get_data_set_from_midi(path_train, target_str)


...... Get X, Y, samples_length, seq_length, hidden_size ...... : 

	from ./train_data/waltzes,
	target_string : 


...... get samples list ......

	Return 5 file name list.

...... get info : longest sequence length ......

	Longest File name : waltzes_simple_chords_1.mid, sequence length : 391


	debug --------> self.seq_length : 391


	Midi file(=./train_data/waltzes/waltzes_simple_chords_1.mid) successfully loaded. Total 1/8 beats = 390
	Midi file(=./train_data/waltzes/waltzes_simple_chords_2.mid) successfully loaded. Total 1/8 beats = 198
	Midi file(=./train_data/waltzes/waltzes_simple_chords_3.mid) successfully loaded. Total 1/8 beats = 198
	Midi file(=./train_data/waltzes/waltzes_simple_chords_6.mid) successfully loaded. Total 1/8 beats = 294
	Midi file(=./train_data/waltzes/waltzes_simple_chords_7.mid) successfully loaded. Total 1/8 beats = 390


In [6]:
# model build: Sequential Model()
#
print "\n...... Start Making Model ......"

#
# model 1 : LSTM
#
model = Sequential()

model.add(LSTM(hidden_size, input_dim=hidden_size, input_length=seq_length, activation='tanh', return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(hidden_size, input_dim=hidden_size, input_length=seq_length, activation='tanh', return_sequences=True))
model.add(Dropout(0.1))
model.add(Dense(X.shape[2]))                        # hidden size
model.add(Activation('softmax'))                    # softmax layer



...... Start Making Model ......


/home/eko/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:10: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
/home/eko/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:10: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(129, return_sequences=True, activation="tanh", input_shape=(391, 129))`
/home/eko/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:12: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
/home/eko/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:12: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(129, return_sequences=True, activation="tanh", input_shape=(391, 129))`


In [7]:
# show model
ModelUtil.layer_info(model)

Checking model ...
	layer shapes
		[00] LSTM                           (, 391, 129)                   (, 391, 129)                  
		[01] Dropout                        (, 391, 129)                   (, 391, 129)                  
		[02] LSTM                           (, 391, 129)                   (, 391, 129)                  
		[03] Dropout                        (, 391, 129)                   (, 391, 129)                  
		[04] Dense                          (, 391, 129)                   (, 391, 129)                  
		[05] Activation                     (, 391, 129)                   (, 391, 129)                  
	layer parameters
		[00] LSTM                           : 133644
		[01] Dropout                        : 0
		[02] LSTM                           : 133644
		[03] Dropout                        : 0
		[04] Dense                          : 16770
		[05] Activation                     : 0
		------------------------------------------------------------
		total             

In [8]:

# weigts value loading
# automatic setting for loading wts weight values
try:
    wts_list = os.listdir(DIR_WEIGHTS)
    if len(wts_list) != 0:
        wts_list.sort()
        model.load_weights(DIR_WEIGHTS + wts_list[-1])
        print "\n...... Loaded weights file : {0} ......".format(wts_list[-1])
except:
    pass


...... Loaded weights file : train_piano_wts_seq_model_2017.05.31.20:00:58.wts ......


In [9]:
##
# set optimizer && loss function
# priority of using optimizer : adam > adadelta > adagrad
print "\n...... Start Compiling Model ......"
model.compile(loss='categorical_crossentropy', optimizer='adam')
# model.compile(loss='categorical_crossentropy', optimizer='adadelta')
# model.compile(loss='categorical_crossentropy', optimizer='adagrad')


...... Start Compiling Model ......


In [10]:
# sample function for prediction.
#
def sample(prob, temperature=1.0):
    # helper function to sample an index from a probability array
    prob = np.log(prob) / temperature
    prob = np.exp(prob) / np.sum(np.exp(prob))
    return np.argmax(np.random.multinomial(1, prob, 1))

##
def sample2(prob, temperature=1.0):
    prob = np.log(prob) / temperature
    prob = np.exp(prob) / np.sum(np.exp(prob))
    # return np.random.multinomial(random.randint(0,5), prob, 1)
    return np.random.multinomial(random.randint(1,5), prob, 1)

##
def choose_note_index(prob, temperature=1.0, max_chords=4):
    # helper function to sample notes from a probability array
    # for now, I make max limitation of chord of 4
    prob = np.log(prob) / temperature
    prob = np.exp(prob) / np.sum(np.exp(prob))
    notes = np.random.choice(prob.shape[0], max_chords, p=prob)
    return np.unique(notes)


def sample_song(seed, midi_file_path):
    print 'Sampling song ...'

    # zero one-of-K matrix
    X_predict = np.zeros((1, seq_length, hidden_size))
    X_predict[0, 0, :] = seed  # save seed

    for i in range(seq_length):
        # prediction
        phat = model.predict(X_predict)
        # note choose
        ix = choose_note_index(phat[0, i])
        # if, end of sequence
        if np.any(ix == 128):
            break
        # saving
        if i < seq_length - 1:
            X_predict[0, i + 1, ix] = 1

    # split before EOS
    total_beats = (X_predict.sum(axis=-1) > 0).sum()
    X_result = X_predict[0, :total_beats]

    # midi file saving
    MidiUtil.save_midi(midi_file_path, X_result)

In [11]:

#
# batch size and epoch value configuration
#
# ================================================================================
training_times = 10
batch_size_num = seq_length
epoch_num = 1
# ================================================================================

In [12]:

time_start_total = datetime.now()
##
print "\n...... Start Training && Predicting with diversity = 1.0 ......"
with open(filename_result_predict, 'a') as file :
    # for iteration in range(1, 100):
    for iteration in range(1, training_times + 1):
        time_start_iteration = datetime.now()

        print '\n'
        print '-' * 100
        print "Iteration : {0} : is started\n".format(iteration)

        time_start_epochs = datetime.now()

        #
        # training start
        #

        # model.fit(X, Y, batch_size=batch_size_num, nb_epoch=epoch_num, show_accuracy=True, shuffle=False)
        model.fit(X, Y, nb_epoch=epoch_num, shuffle=False)

        time_end_epochs = datetime.now()
        print "\n\t{0} Epochs time : {1}".format(epoch_num, time_end_epochs - time_start_epochs)

        # save wts
        filename_wts = DIR_WEIGHTS + "train_piano_wts_seq_model_{0}.wts".format(datetime.now().strftime("%Y.%m.%d.%H:%M:%S"))
        model.save_weights(filename_wts, overwrite=False)
        print "\n\tSaved Weights File : {0}".format(filename_wts)

        #
        # prediction start
        #

        time_start_predict = datetime.now()
        # every 10 iteration
        if iteration % 10 == 0:
            # # for diversity in [0.2, 0.5, 1.0, 1.2]:

            # get first note from training data set
            row = np.random.randint(0, samples_length, 1)
            seed = X[row, X[0].sum(axis=-1) > 0, :][0, :]

            # predicted midi file
            FILE_PRED_MIDI = DIR_PREDICTED_MIDI + "rnn_lstm_pred_midi_{0}.mid".format(datetime.now().strftime("%Y.%m.%d.%H:%M:%S"))
            sample_song(seed, FILE_PRED_MIDI)

            # debug용
            time_end_predict = datetime.now()
            print "\n\tPredict time : {0}".format(time_end_predict - time_start_predict)

        #
        # predict finished
        #

        time_end_iteration = datetime.now()
        print "\n\t{0}th Iteration time : {1}".format(iteration, time_end_iteration - time_start_iteration)

        time_passed_total = datetime.now() - time_start_total
        print "\n\tPassed time from starting Training Process : {0}".format(time_passed_total)

        # log 저장
        # save_str =""
        # file.write(save_str)
##



...... Start Training && Predicting with diversity = 1.0 ......


----------------------------------------------------------------------------------------------------
Iteration : 1 : is started



/home/eko/anaconda2/lib/python2.7/site-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
5/5 [==============================] - 0s - loss: 6.2786

	1 Epochs time : 0:00:02.084358

	Saved Weights File : ./wts/train_piano_wts_seq_model_2017.05.31.20:31:34.wts

	1th Iteration time : 0:00:02.113041

	Passed time from starting Training Process : 0:00:02.114631


----------------------------------------------------------------------------------------------------
Iteration : 2 : is started

Epoch 1/1
5/5 [==============================] - 0s - loss: 6.0265

	1 Epochs time : 0:00:00.478280

	Saved Weights File : ./wts/train_piano_wts_seq_model_2017.05.31.20:31:35.wts

	2th Iteration time : 0:00:00.485136

	Passed time from starting Training Process : 0:00:02.599918


----------------------------------------------------------------------------------------------------
Iteration : 3 : is started

Epoch 1/1
5/5 [==============================] - 0s - loss: 5.6702

	1 Epochs time : 0:00:00.476591
[WARNING] ./wts/train_piano_wts_seq_model_2017.05.31.20:31:35.wts already exists